# TO DO

1) extract more relevant data points

    1.1) Loop through all offers and extract possible salary keys (avantages, etc...)
    1.2) Identify other data points to be retrieved and same idea

2) parse dict data from columns (before conversion to DF? after conversion to DF? identify whats most efficient)

3) identify latest offer published (max date)

    3.1) store max date as variable
    3.2) store max date as latest offer fetched in SQL database table
    3.3) retrieve latest offer date from SQL database table
        if exists fetch it, if not then start date is 30 days ago

4) create a data model for data storage

    4.1) list target tables
    4.2) list relationships between tables
    4.3) list pk/fk/constraints
    4.4) create a json file with all of these requirements
        4.5) for things such as salary, make it dynamic so columns can be added if needed?
    4.5) have dynamic SQL written for table creations + table updates

4) create an airflow project that runs on demand: runs the script, adds records to the DB, uploads to PostgreSQL database


# DONE



# Connection a l'API France Travail

Process:
1) Request a token for the API you want to work with
2) Retrieve the token
3) Use it to communicate with your target API

# Imports

In [9]:
# Import required libraries
import os
import requests
from requests.exceptions import HTTPError
from datetime import datetime, timedelta
import pandas as pd
import sqlite3

# Identifiers

In [10]:
# reading env variables
client_id = os.getenv("FT_API_CLIENT_ID")
client_secret = os.getenv("FT_API_CLIENT_SECRET")

# Getting an API access token

In [11]:
# URL to get an access token
# specify the realm at the end of the endpoint
url = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"

# POST request parameters
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "api_offresdemploiv2 o2dsoffre",
}

# Headers
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "francetravail.io"
}

# API request
response = requests.post(
    url, 
    data=data, 
    headers=headers
)

# if positive response, store token
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Token successfully granted:", access_token)
else:
    print("Error when retrieving the API token:", response.text)

Token successfully granted: _IBYBo-nOFhLhr2lxUpCDyqyjd0


# Connecting to a SQLite3 database

In [12]:
conn = sqlite3.connect('airflow.db')
cursor = conn.cursor()

In [13]:
# casting dates to isoformat
min_creation_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ") # 30 days ago
max_creation_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") # today

# API endpoint
offres_emplois_url = "https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"

# headers
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}

# parameters
# looking for
    # data related
    # in two specific departments (Paris and Hauts de Seine)
    # permanent contract (CDI)
    # created in the past 90 days
    # senior experience (3 is the max value = over 3 years of XP)
params = {
    "motsCles": "data",
    "departement": "75,92",
    "typeContrat": "CDI",
    "minCreationDate": str(min_creation_date),
    "maxCreationDate": str(max_creation_date),
    "experience": "3",
}

try:
    response = requests.get(
        offres_emplois_url, 
        params=params, 
        headers=headers
    )
    response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")

offres = response.json()

In [14]:
# creates a dataframe with the keys below (considered relevant)

selected_keys = [
    "intitule", 
    "description", 
    "dateCreation",
    "appellationlibelle",
    "secteurActiviteLibelle",
    "typeContratLibelle",
    "salaire"
]

filtered_list = [
    {key: offre[key] for key in selected_keys if key in offre}
    for offre in offres["resultats"]
]

offres_df = pd.DataFrame(filtered_list)

In [15]:
# some data processing
offres_df["dateCreation"] = pd.to_datetime(offres_df["dateCreation"]).dt.date

# get the max creation date for offers
max_date = offres_df["dateCreation"].max()

In [20]:
offres_df.columns

Index(['intitule', 'description', 'dateCreation', 'appellationlibelle',
       'secteurActiviteLibelle', 'typeContratLibelle', 'salaire'],
      dtype='object')

In [ ]:
# filters on row 1, column salaire value and shows the data
offres_df.iloc[3, 6]

In [ ]:
# filters on row 1, column salaire value and shows the data




offres_df.iloc[1, "salaire"]

In [ ]:
# parses the data in salaire column that is a json/dict structure


# WORK IN PROGRESS

In [ ]:
# let's check the different keys in a job offer
# we will analyze a single offer and retrieve all of its keys
keys = list(offres["resultats"][0].keys())

for key in keys:
    print(key)